In [ ]:
import h5py
import rotation_mio as rot
import time_conversion as tiempo
import bines2
import density as den
import circular_velocity as vcirc
import numpy as np


h=0.732
G = 4.299e-6
a0=1
H0 = h*100
omega_lambda=0.716
omega_matter=0.1277/(h**2.)
omega0 = omega_lambda + omega_matter

vector3 = ('M31','MW','r','M33')

path = '/home/omarioni/'
atime = np.loadtxt(path + 'redshift_outputs.txt')
aexp = atime[:,2]


path = '/mnt/is2/alejandro/ornella/'
snap = h5py.File(path+'outputs_1/snap_496.h5py', 'r')

snapshot=range(496,50,-1) #SNAPSHOTS

path2 = '/home/omarioni/Barras_GdGs/Barras_Gd/_data/Toomre/'

halos = (0,1,3)
for isnap in snapshot:
# isnap=496
    snap = h5py.File(path + 'outputs_1/snap_'+str('%03d'%isnap)+'.h5py', 'r')

    for i in halos:

        cm   = snap['subhalo_'+ str('%03d' %i)+ '/Center'][()]
        r200 = snap['subhalo_'+ str('%03d' %i)+ '/R200'][()]

        pdrk = snap['subhalo_'+ str('%03d' %i) + '/Drk/Coordinates'][()]
        mdrk = snap['subhalo_'+ str('%03d' %i) + '/Drk/Masses'][()]
        vdrk = snap['subhalo_'+ str('%03d' %i) + '/Drk/Velocities'][()]
        
        z = a0/aexp[isnap] - 1.
        Ht = H0*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)
        time  = tiempo.conv(z, h, omega_lambda, omega_matter)
        
        
        xdrk = (pdrk[:,0]-cm[0])*aexp[isnap]/h
        ydrk = (pdrk[:,1]-cm[1])*aexp[isnap]/h
        zdrk = (pdrk[:,2]-cm[2])*aexp[isnap]/h

        v_x = vdrk[:,0] *np.sqrt(aexp[isnap]) + Ht *xdrk/1000.
        v_y = vdrk[:,1] *np.sqrt(aexp[isnap]) + Ht *ydrk/1000.
        v_z = vdrk[:,2] *np.sqrt(aexp[isnap]) + Ht *zdrk/1000.

    #----------------------masas----------------------------
        mdrk = mdrk/h

        r200 = r200*aexp[isnap]/h

    #-------------------------------------------------------------------            
        corte,=np.where(r<3.)
        veloc,=np.where(r<r50)

    #----------componentes de la velocidad del centro de masa------------
        vxcm = sum(mstr[veloc]*v_x[veloc])/sum(mstr[veloc])
        vycm = sum(mstr[veloc]*v_y[veloc])/sum(mstr[veloc])
        vzcm = sum(mstr[veloc]*v_z[veloc])/sum(mstr[veloc])

    #----- velocidades de las estrellas respecto del centro de masa de la galaxia---------
        vx = v_x - vxcm
        vy = v_y - vycm
        vz = v_z - vzcm

        e1x,e2x,e3x,e1y,e2y,e3y,e1z,e2z,e3z = rot.rot1(mstr,xstr,ystr,zstr,vx,vy,vz,3.*aexp[isnap])

        xn = e1x*xstr + e1y*ystr + e1z*zstr
        yn = e2x*xstr + e2y*ystr + e2z*zstr
        zn = e3x*xstr + e3y*ystr + e3z*zstr
        vxn = e1x*vx + e1y*vy + e1z*vz
        vyn = e2x*vx + e2y*vy + e2z*vz
        vzn = e3x*vx + e3y*vy + e3z*vz

        rn = np.sqrt(xn**2 + yn**2 + zn**2)
        

    #----------calculamos las dispersiones de velocidades---------------------
        corte1, = np.where(rn < rgal)
        #     corte_bin2, = np.where(rn < 6.)

        R = np.sqrt(xn[corte1]**2 + yn[corte1]**2)

        nbin = 100 #20
        rbin, nodos = bines2.rbin1(R, nbin)

        Vr = (xn[corte1]*vxn[corte1] + yn[corte1]*vyn[corte1])/R
        Vz = vzn[corte1]
        Vt = (xn[corte1]*vyn[corte1] - yn[corte1]*vxn[corte1])/R

        sigma_z = np.zeros(nbin)
        sigma_R = np.zeros(nbin)
        sigma_t = np.zeros(nbin)


        for j in range(nbin):
            mask, = np.where((nodos[j] < R) & (nodos[j+1] > R))
            n = len(mask)
            Vz_mean = np.mean(Vz[mask])
            Vr_mean = np.mean(Vr[mask])
            Vt_mean = np.mean(Vt[mask])

            sigma_z[j] = np.sqrt(sum((Vz[mask] - Vz_mean)**2)/n)
            sigma_R[j] = np.sqrt(sum((Vr[mask] - Vr_mean)**2)/n)
            sigma_t[j] = np.sqrt(sum((Vt[mask] - Vt_mean)**2)/n)


        surf_den, rden = den.surf_density(R,mstr[corte1]*1e10,nbin)

        xngas = e1x*xgas + e1y*ygas + e1z*zgas
        yngas = e2x*xgas + e2y*ygas + e2z*zgas
        zngas = e3x*xgas + e3y*ygas + e3z*zgas
        rgas  = np.sqrt(xngas**2 + yngas**2 + zngas**2)
        Rgas = np.sqrt(xngas**2 + yngas**2)

        xndrk = e1x*xdrk + e1y*ydrk + e1z*zdrk
        yndrk = e2x*xdrk + e2y*ydrk + e2z*zdrk
        zndrk = e3x*xdrk + e3y*ydrk + e3z*zdrk
        rdrk  = np.sqrt(xndrk**2 + yndrk**2 + zndrk**2)
        Rdrk = np.sqrt(xndrk**2 + yndrk**2)

        corte_gas = np.where(rgas < rgal)
        corte_drk = np.where(rdrk < rgal)
    
        Rtot= np.concatenate([R, Rgas[corte_gas], Rdrk[corte_drk]])
        mtot= np.concatenate([mstr[corte1], mgas[corte_gas], mdrk[corte_drk]])

        vc, r_vc = vcirc.Vc_bin(Rtot, mtot*1e10, nbin)

        Q = tommre(rbin, vc, sigma_R, surf_den)
    
        data = np.ndarray([nbin,7])
        data[:,0] = rbin
        data[:,1] = sigma_z
        data[:,2] = sigma_R
        data[:,3] = sigma_t
        data[:,4] = surf_den
        data[:,5] = vc
        data[:,6] = Q
    
        np.savetxt(path2 + str('%s'%vector3[i])+'GD_'+str('%03d'%isnap)+'_toomre.dat',data, fmt='%12.6f')
    
    